In [ ]:
from dotenv import load_dotenv
import os
from hera.workflows import EmptyDirVolume, models, script, Artifact, Parameter, DAG, Steps, Step, NoneArchiveStrategy, Workflow
from hera.shared import global_config

load_dotenv("/home/otto/s1_zarr/.env")

In [ ]:
global_config.host = "https://dev.services.eodc.eu/workflows/"
global_config.namespace = "inca"
global_config.token = os.getenv("argo_token")
global_config.image = "ghcr.io/oscipal/image_zarr:latest"

In [ ]:
@script(outputs=Artifact(name="output-artifact", path="/tmp/INCA.nc", archive=NoneArchiveStrategy()))
def create_artifact():
    from urllib.request import urlretrieve
    import datetime
    import os 

    artifact_path = "/tmp"

    url = f"https://public.hub.geosphere.at/datahub/resources/inca-v1-1h-1km/filelisting/RR/INCAL_HOURLY_RR_202506.nc"
    urlretrieve(url, os.path.join(artifact_path, "INCA.nc"))

In [ ]:
@script(inputs=Artifact(name="input-artifact", path="/tmp/INCA.nc"))
def consume_artifact():
    import xarray as xr

    artifact_path="/tmp/INCA.nc"

    ds = xr.open_dataset(os.path.join(artifact_path, "INCA.nc")).isel(time=1, x=1, y=1)["RR"].load()
    print(ds.values)

In [ ]:
with Workflow(
    generate_name="artifact-storage-",
    entrypoint="workflow"
) as w:
    with DAG(name="workflow"):
        download = create_artifact()
        consume = consume_artifact(arguments=[download.get_artifact("output-artifact").with_name("input-artifact")])

        download >> consume

In [ ]:
w.create()